In [ ]:
dir = '/data/HCP/HCP_1200/download_swarm/**/*'
if True:
#     -lu gives access time
#     -d1 gives just the file/dir instead of the contents
#     the glob pattern provides all the files/dirs
#     the awk command turns it into tab separated output
    !ls -lu -d1  --time-style long-iso {dir} | awk -v OFS="\t" '$1=$1' > files_list.txt
    

In [ ]:
import pandas as pd
cols = ['perms','links','user','group','size','year','time','dir']
df  =  pd.read_csv('files_list.txt',sep = '\t',header=None,names=cols)
df